In [112]:
import pandas as pd

GAMES = ["J03WQQ", "J03WR9", "J03WOH", "J03WOY", "J03WPY"]
PLAYERS = ["home_11", "home_19", "home_23", "home_25", "home_5", "home_7", "home_9"]
full_df = pd.read_csv("output/full_das.csv")
full_df_ohne_cut = full_df[
    (full_df["game_id"] == GAMES[0]) &
    (full_df["player_id"] == PLAYERS[3]) &
    (full_df["step"] == "step125") &
    (~full_df["method"].str.contains("cut", case=False, na=False)) &
    (full_df["method"] == "random")
].reset_index(drop=True)
full_df_mit_cut = full_df[
    (full_df["game_id"] == GAMES[0]) &
    (full_df["player_id"] == PLAYERS[3]) &
    (full_df["step"] == "step125") &
    (full_df["method"].str.contains("cut", case=False, na=False))
].reset_index(drop=True)

In [85]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def plot_player_time_series(df_input, metric_prefix="PotentialDAS_", title=None, max_traces=40, height=500, width=1000):
    df = df_input.copy()
    
    # df.reset_index(inplace=True)
    # df.rename(columns={"index": "frame"})
    x = np.arange(len(df))
    das_cols = [c for c in df.columns if c.startswith(metric_prefix)]
        
    fig = go.Figure()
    
    for col in das_cols:
        label = col[len(metric_prefix):]
        y = df[col].values
        
        hovertemplate = (
            "frame_index: %{x}<br>"
            f"{label}<br>"
            f"DAS_potential: " + "%{y:.3f}<extra></extra>"
        )
        fig.add_trace(
            go.Scatter(
               x=x, y=y,
               mode="lines+markers",
               name=label,
               hovertemplate=hovertemplate,
               marker=dict(size=6),
               line=dict(width=1)
            )
        )
    
    if title is None:
        title = "DAS_potential Zeitverlauf (indexbasiert)"
    fig.update_layout(
        title=title,
        xaxis_title="Frame (indexbasiert, gleichabständig)",
        yaxis_title="DAS_potential",
        height=height,
        width=width,
        legend_title="Methode | Parameter",
        hovermode="x unified"
    )
    # fig.show()
    return fig

def add_frame_ranks(df_input, metric_keyword="PotentialDAS", rank_prefix="Rank_"):
    
    df = df_input.copy().reset_index(drop=True)
    
    das_cols = [c for c in df.columns if metric_keyword in str(c)]
    das_df = df[das_cols].astype(float)
    row_ranks = das_df.rank(axis=1, method="average", ascending=False)

    rank_cols = [f"{rank_prefix}{c}" for c in das_cols]
    row_ranks.columns = rank_cols

    
    result = pd.concat([df, row_ranks], axis=1)
    return result


In [114]:
df = full_df_ohne_cut.copy()
keys = ["game_id", "player_id", "frame"]

pivot = df.pivot_table(
    index=keys,
    columns="tuning",
    values=["DAS_new", "DAS_potential"],
    aggfunc="first"
)
pivot.index = [frame for _, _, frame in pivot.index]
pivot.columns = [f"{metric.split("_")[1].capitalize()}DAS_{tuning.split("-")[0]}-{tuning.split("-N_")[1]}" for metric, tuning in pivot.columns]
pivot.columns = [col.split("-")[0] if "all_positions" in col else col for col in pivot.columns]
pivot_ranked = add_frame_ranks(pivot)

fig = plot_player_time_series(pivot)
fig.show()

fig = plot_player_time_series(pivot_ranked, metric_prefix="Rank_PotentialDAS_")
fig.show()

In [88]:
df = full_df_mit_cut.copy()
keys = ["game_id", "player_id", "frame"]

pivot = df.pivot_table(
    index=keys,
    columns="tuning",
    values=["DAS_new", "DAS_potential"],
    aggfunc="first"
)
pivot.index = [frame for _, _, frame in pivot.index]
pivot.columns = [f"{metric.split("_")[1].capitalize()}DAS_{tuning.split("-")[0]}-{tuning.split("-N_")[1]}" for metric, tuning in pivot.columns]
pivot.columns = [col.split("-")[0] if "all_positions" in col else col for col in pivot.columns]
pivot_ranked = add_frame_ranks(pivot)

fig = plot_player_time_series(pivot)
fig.show()
fig = plot_player_time_series(pivot_ranked, metric_prefix="Rank_PotentialDAS_")
fig.show()

In [111]:
keys = ["game_id", "player_id", "frame"]
df_100_all = full_df[
    (full_df["tuning"].str.contains("N_100")) |
    (full_df["method"].str.contains("all_positions"))
]
pivot_100_all = df_100_all.pivot_table(
    index=keys,
    columns="tuning",
    values=["DAS_new"],
    aggfunc="first",
    fill_value=0
)
pivot_100_all.columns = [f"{metric.split("_")[1].capitalize()}DAS_{tuning.split("-")[0]}-{tuning.split("-N_")[1]}" for metric, tuning in pivot_100_all.columns]
pivot_100_all.columns = [col.split("-")[0] if "all_positions" in col else col for col in pivot_100_all.columns]
pivot_100_all["Diff ohne Cut"] = pivot_100_all["NewDAS_all_positions"] - pivot_100_all["NewDAS_random-100"]
pivot_100_all["Diff mit Cut"] = pivot_100_all["NewDAS_all_positions_cut_1.0"] - pivot_100_all["NewDAS_random_cut_1.0-100-C_1.0"]
pivot_comp = pivot_100_all[["Diff ohne Cut", "Diff mit Cut"]]
display(pivot_comp["Diff mit Cut"].unique())
display(pivot_comp["Diff ohne Cut"].unique())

array([0.])

array([0.])